### Read file data

In [ ]:
import pandas as pd

queries_train = pd.read_csv('/home/luungoc/BTL-2023.1/Deep learning/collectionandqueries/collection.tsv', sep='\t')

In [ ]:
from tqdm import tqdm

collention = []

for i in tqdm(range(len(queries_train))):
    collention.append({
        'PID': queries_train.iloc[i][0],
        'Description': queries_train.iloc[i][1]
    })

In [ ]:
from tqdm import tqdm

new_data = []
for item in tqdm(qrel_train):
    qid, pid = item['QID'], item['PID']
    
    new_data.append({
        'QID': qid,
        'PID': pid,
        'query': queries_train[qid]['PID'],
        'passage': collention[int(pid)]['Description']
    })

In [ ]:
import csv
import json

# Specify the input TSV file and output JSON file
tsv_file = '/home/luungoc/BTL-2023.1/Deep learning/collectionandqueries/qrels.train.tsv'



data = []
with open(tsv_file, 'r', encoding='utf-8', newline='') as tsvfile:
    reader = csv.DictReader(tsvfile, delimiter='\t')
    for row in reader:
        data.append(row)

new_data = []

for item in data:
    new_data.append({
        'QID': item['QID'],
        'PID': item[None][1]
    })

with open(tsv_file.replace('tsv', 'json'), 'w', encoding='utf-8') as jsonfile:
    json.dump(new_data, jsonfile, indent=2)

print(f"Conversion complete. JSON file saved as {tsv_file.replace('tsv', 'json')}")


### Test dataloader

In [ ]:
from src.dataloader import MyDataset
from torch.utils.data import DataLoader

data_loader = MyDataset(data_path='/home/luungoc/BTL-2023.1/Deep learning/Data Json/data_total.json',
                        tokenizer='bert-base-uncased',
                        max_length=256)


train_data = DataLoader(dataset=data_loader, batch_size=8, num_workers=0)
data_loader[1]

In [ ]:
import pickle
import json


query_train = json.load(open('/home/luungoc/BTL-2023.1/Deep learning/src/Data Json/query_train.json', 'r')) 
collection = json.load(open('/home/luungoc/BTL-2023.1/Deep learning/src/Data Json/collection.json', 'r')) 

In [ ]:
import pickle

# Mở file Pickle để đọc
with open('/home/luungoc/BTL-2023.1/Deep learning/src/Data Json/train_qrels_first_100k_with_negatives.pkl', 'rb') as file:
    # Sử dụng pickle.load để đọc dữ liệu từ file
    pair_train = pickle.load(file)

In [ ]:


from tqdm import tqdm
new_data = []

for item in tqdm(pair_train):
    temp = {}
    temp['negative_text'] = []
    for itm in item['hard_negative_pids'][:5]:
        temp['negative_text'].append({
            'Pid': itm,
            'P_text': collection[itm]
        })
    temp['query_text'] = {
        'Qid': item['qid'],
        'Q_text': query_train[str(item['qid'])]['PID'],
    }
    temp['positive_text'] = {
        'Pid' : item['pid'],
        'P_text': collection[item['pid']]
    }
    new_data.append(temp)
#     print(item['query_text'])

In [ ]:
import torch.optim as optim
import torch
import argparse
import torch.nn as nn
from torch.utils.data import DataLoader
from model.cross_encoder import DoubleBert
from train_v2 import MyTrainer
from config import Configs
from dataloader import MyDataset


class args:
    config = 'config.ini'
    
config = Configs

# data = MyDataset(config=config)

# train_data = DataLoader(dataset=data, batch_size=2, num_workers=2, shuffle=True)
# model = DoubleBert(config=config)

In [ ]:
model = DoubleBert(config=config)

state_dict = torch.load('/home/luungoc/BTL-2023.1/Deep learning/Checkpoint/Adapter/Step-3200.pth', map_location=torch.device('cpu') )
state_dict = {key.replace('module.', ''): value for key, value in state_dict.items()}

model.load_state_dict(state_dict, strict=False)

model(query, pos)

In [ ]:

query = 'My name is Ngoc'
pos = 'Sky blue'

model(query, pos)

In [ ]:
import torch

a = torch.tensor([[7.6680]])
a.item()


In [ ]:
import torch
import torch.nn as nn

class CrossEntropyLoss(nn.Module):
    def __init__(self):
        super(CrossEntropyLoss, self).__init__()

    def forward(self, softmax_positive, label_positive, softmax_negative, label_negative):
        # Chuyển đổi thành tensor và chuyển đổi thành kiểu Long
        label_positive = torch.tensor(label_positive, dtype=torch.long)
        label_negative = torch.tensor(label_negative, dtype=torch.long)

        # Tính toán CrossEntropy loss cho positive
        positive_loss = nn.CrossEntropyLoss()(softmax_positive, label_positive)

        # Tính toán CrossEntropy loss cho negative
        negative_loss = nn.CrossEntropyLoss()(softmax_negative, label_negative)

        # Tính trung bình loss
        mean_loss = (positive_loss + negative_loss) / 2

        return mean_loss

# Ví dụ sử dụng CrossEntropyLoss
ce_loss_criterion = CrossEntropyLoss()

# Giả sử bạn có các giá trị như mô tả
label_positive = [1, 1, 1, 1]
label_negative = [0, 0, 0, 0]
softmax_positive = torch.tensor([[0.1, 0.9], [0.2, 0.8], [0.3, 0.7], [0.25, 0.75]])
softmax_negative = torch.tensor([[0.1, 0.9], [0.2, 0.8], [0.3, 0.7], [0.25, 0.75]])

# Tính loss
mean_ce_loss = ce_loss_criterion(softmax_positive, label_positive, softmax_negative, label_negative)
print("Mean CrossEntropy Loss:", mean_ce_loss.item())


In [ ]:
import torch.optim as optim
import torch
import argparse
import torch.nn as nn
from torch.utils.data import DataLoader
from model.cross_encoder import DoubleBert
from train_v2 import MyTrainer
from config import Configs
from dataloader import MyDataset


class args:
    config = 'config.ini'
    
config = Configs

data = MyDataset(config=config)

train_data = DataLoader(dataset=data, batch_size=2, num_workers=2, shuffle=True)
model = DoubleBert(config=config)

trainable_params_count = 0

no_decay = ["adapter.proj_up.bias", "adapter.proj_down.bias", "LayerNorm.bias"]
classifier = ["linear_layer.weight", "linear_layer.bias", "normalize.weight", "normalize.bias", "last_output.weight", "last_output.bias"]
layers = ["adapter.proj_up.weight", "adapter.proj_down.weight", "LayerNorm.weight"]

layers.extend(classifier)
# no_decay.extend(cls_bias)   

optimizer = [
    
    {
        "params": [p for n, p in model.named_parameters() if any([nd in n for nd in layers])],
        "weight_decay": 0.1,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },

]


for group in optimizer:
    for param in group["params"]:
        trainable_params_count += param.numel()
print(f'Total Trainable params: {trainable_params_count}')

In [ ]:
for n, p in model.named_parameters():
    print(n)

In [ ]:
import pickle
import json

with open('/home/luungoc/BTL-2023.1/Deep learning/Data Json/dev_qrels_first_10k_with_bm25_top1000.pkl', 'rb') as file:
    pair_train = pickle.load(file)
    
# collection = json.load(open('/home/luungoc/BTL-2023.1/Deep learning/Data Json/collection.json', 'r'))
# query = json.load(open('/home/luungoc/BTL-2023.1/Deep learning/Data Json/queries.train.json', 'r'))


In [ ]:
query_dev = json.load(open('/home/luungoc/BTL-2023.1/Deep learning/Data Json/queries.dev.json', 'r'))
abc = json.load(open('/home/luungoc/BTL-2023.1/Deep learning/src/data_test.json', 'r'))

In [ ]:
pair_train[1]['qid']

In [ ]:
def sort_list_of_dicts(list_of_dicts):
    sorted_list = sorted(list_of_dicts, key=lambda x: x['QID'])
    return sorted_list


aa = sort_list_of_dicts(query_dev)

In [ ]:
len(aa)

In [ ]:
def search_qid(list_of_dicts, target_qid):
    left, right = 0, len(list_of_dicts) - 1

    while left <= right:
        mid = (left + right) // 2
        mid_qid = list_of_dicts[mid]["QID"]

        if mid_qid == target_qid:
            return list_of_dicts[mid]["PID"]
        elif mid_qid < target_qid:
            left = mid + 1
        else:
            right = mid - 1

    return None

# Ví dụ sử dụng
list_of_dicts = [
    {"QID": "123", "PID": "abc"},
    {"QID": "456", "PID": "def"},
    {"QID": "789", "PID": "ghi"},
    {"QID": "999", "PID": "jkl"},
]

target_qid = "456"
result_pid = search_qid(list_of_dicts, target_qid)

if result_pid is not None:
    print(f"QID {target_qid} được tìm thấy, PID tương ứng là {result_pid}")
else:
    print(f"QID {target_qid} không được tìm thấy trong danh sách.")


In [ ]:
collection[7068757]['Description']

In [ ]:
pair_train[101].keys()

In [ ]:
import json

query = json.load(open('/home/luungoc/BTL-2023.1/Deep learning/src/result.json', 'r'))



In [ ]:
query[1]['query']

In [ ]:

for sample in tqdm(query):
    with torch.no_grad():
        query = sample['query']
        positive = sample['positive']['text']
        id_pos = sample['positive']['id']
        
        sim, soft = model(query, positive)
        
        list_eval.append({
            'id': id_pos,
            'sim': sim.item()
        })
        
        for item in sample['negative']:
            text = item['text']
            id_neg = item['id']

            sim, soft = model(query, text)
            
            list_eval.append({
                'id': id_neg,
                'sim': sim.item()
            })
            
json.dump(list_eval, open('/home/luungoc/BTL-2023.1/Deep learning/src/result.json', 'w'), ensure_ascii=False, default=numpy_encoder)

In [ ]:
import pickle

with open('/home/luungoc/BTL-2023.1/Deep learning/Data Json/base_biencoder-scores.pkl', 'rb') as file:
    pair_train = pickle.load(file)

In [1]:
import pickle
import json

with open('/home/luungoc/BTL-2023.1/Deep learning/Data Json/base_biencoder-scores.pkl', 'rb') as file:
    base_encoder = pickle.load(file)
    
collection = json.load(open('/home/luungoc/BTL-2023.1/Deep learning/Data Json/dev_qrels_with_text_bm25_top100.json', 'r'))

In [2]:
collection[1]

{'query': {'id': 1102431,
  'text': 'why did rachel carson write an obligation to endure'},
 'positive_passage': {'id': 7066866,
  'text': "The Obligation to Endure by Rachel Carson Rachel Carson's essay on The Obligation to Endure, is a very convincing argument about the harmful uses of chemicals, pesticides, herbicides, and fertilizers on the environment."},
 'bm25_top100': [{'id': 6915470,
   'text': "An obligation is a duty. It's your obligation or responsibility to call your mom on Mother's Day. Sending flowers would be even better. When you are morally or legally bound to a particular commitment, it's your obligation to follow through on it. If you see a crime taking place, for example, it's your obligation to notify the police. If an elderly person comes onto a full bus, it's your obligation to give up your seat for him."},
  {'id': 2801136,
   'text': "Ok well here is the rest of the story. Elliot Lurie did not write the song Brandy. He did not write the lyrics or the music. Th